In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

import torch.utils.data as data 
import torch
from torch_geometric.data import Data
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch.utils.data import Dataset, DataLoader

import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator #PygNodePropPredDatase

from IPython.display import clear_output


from sklearn import preprocessing

In [2]:
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges_init = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

# Change column names of df_features
colNames1 = {'0': 'txId', 1: "Time step"}
colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(93)}
colNames3 = {str(ii+95): "Aggregate_feature_" + str(ii+1) for ii in range(72)}

colNames = dict(colNames1, **colNames2, **colNames3 )
colNames = {int(jj): item_kk for jj,item_kk in colNames.items()}

df_features = df_features.rename(columns=colNames)

In [3]:
# Specify the path to your pickle file
pickle_file_path = 'C:\\Users\\gsamp\\OneDrive\\Documents\\AI-3\\2n Semestre\\Projecte de Síntesi 2\\GraphAnomaly\\elipticData_graph.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as f:
    # Load the data from the pickle file
    G = pkl.load(f)


In [4]:
df_merged = pd.merge(left=df_features, right=df_classes, on="txId", how="left")
df_merged = df_merged.set_index('txId')

class_mapping = {'unknown': 2, '1':0, '2':1}
df_merged['class'] = df_merged['class'].replace(class_mapping)

df_merged = df_merged.reset_index()

df_merged = df_merged.loc[df_merged['class'].isin([0,1])]
df_merged = df_merged.reset_index(drop=True)
print(df_edges_init.shape)
df_edges = df_edges_init.loc[((df_edges_init['txId1'].isin(df_merged['txId'])) & (df_edges_init['txId2'].isin(df_merged['txId'])))]
df_edges = df_edges.reset_index(drop=True)
print(df_edges.shape)

C:\Users\gsamp\AppData\Local\Temp\ipykernel_12932\183557646.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged['class'] = df_merged['class'].replace(class_mapping)


(234355, 2)
(36624, 2)


In [5]:
mapping_txid = dict(zip(df_merged['txId'], list(df_merged.index)))

m = True
if m:
    df_edges_mapped = df_edges.replace({'txId1': mapping_txid, 'txId2': mapping_txid})
    df_edges_mapped.to_pickle('dades_guillem\mapped_edges.pkl')
else:
    df_edges_mapped = pd.read_pickle('dades_guillem\mapped_edges.pkl')

In [6]:
x = torch.tensor(df_merged.drop(columns=['class', 'Time step']).values, dtype=torch.float)
edge_index = torch.tensor(df_edges_mapped.values, dtype=torch.long).T
y = torch.tensor(df_merged['class'].values)
time = torch.tensor(df_merged['Time step'].values)


data = Data(x=x, edge_index=edge_index, y=y, time=time)


test_size = 0.2  # 20% of the data will be in the test set

# Split data into train and test sets
train_idx, test_idx = train_test_split(range(len(y)), test_size=test_size, random_state=42)

# Create train and test masks
train_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True

data.train_mask = train_mask
data.test_mask = test_mask

In [7]:
#prepare data for the AE case

#separate illicit and licit data
licit_x = torch.tensor(df_merged.loc[df_merged['class'] == 1].drop(columns=['class', 'Time step']).values, dtype=torch.float)
licit_y = torch.tensor(df_merged.loc[df_merged['class'] == 1]['class'].values)
licit_data = Data(x=licit_x, y=licit_y)

illicit_x = torch.tensor(df_merged.loc[df_merged['class'] == 0].drop(columns=['class', 'Time step']).values, dtype=torch.float)
illicit_y = torch.tensor(df_merged.loc[df_merged['class'] == 0]['class'].values)
illicit_data = Data(x=illicit_x, y=illicit_x)

#prepare train/test split oly with the licit data which will be used to train the AE
test_size = 0.2
#the real target of the AE won't be the y però aquí només ens importa el tamany (xd)
train_idx, test_idx = train_test_split(range(len(licit_y)), test_size=test_size, random_state=42)

# Create train and test masks
train_mask = torch.zeros(len(licit_y), dtype=torch.bool)
test_mask = torch.zeros(len(licit_y), dtype=torch.bool)

train_mask[train_idx] = True
test_mask[test_idx] = True

#declarar-los com a mètodes per fer-ho més còmode
licit_data.train_mask = train_mask
licit_data.test_mask = test_mask

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(sample, dtype=torch.float32)

# # Define batch size
# batch_size = 64
# sampling_idxs = torch.randperm(licit_data.x.size(0))[:2000]
# dataset = CustomDataset(licit_data.x[sampling_idxs])
# data_loader = DataLoader(dataset, batch_size=batch_size)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=50):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.Tanh(),
            nn.Linear(100, 50),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 100),
            nn.Tanh(),
            nn.Linear(100, input_dim),
            nn.ReLU()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded



In [10]:
def train_ae(autoencoder, data_loader, criterion, optimizer, val_loader, num_epochs=10, learning_rate=0.001):   
    total_train_loss = [] 
    total_validation_loss = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in data_loader:
            optimizer.zero_grad()
            
            # Forward pass
            outputs = autoencoder(inputs)
            
            # Compute the loss
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        train_loss = running_loss / len(data_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}")
        validation_loss = val_ae(autoencoder=autoencoder, data_loader=val_loader, criterion=criterion)
        print(f"Validation Loss: {validation_loss:.4f}")
        total_train_loss.append(train_loss)
        total_validation_loss.append(validation_loss)
    return total_train_loss, total_validation_loss

def val_ae(autoencoder, data_loader, criterion):
    autoencoder.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = autoencoder(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
    epoch_loss = val_loss / len(data_loader.dataset)
    return epoch_loss


In [11]:
batch_size = 256
n= 2000
#samplejar només n elements de les lícites per entrenar el AE
x = torch.tensor(df_merged.loc[df_merged["class"] == 1].drop(columns=['class', 'txId', 'Time step'])[:n].values)
#amb el scaler la loss passe de 4 a 0.1
x = preprocessing.MinMaxScaler().fit_transform(x)

train_data, val_data = train_test_split(x, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(val_data)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

#hiperparàmetres
lr = 0.001
EPOCHS = 50

input_dim = x.shape[1]
model = Autoencoder(input_dim)
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

training_loss, validation_loss = train_ae(autoencoder=model, data_loader=train_loader, criterion=criterion, optimizer=optimizer, val_loader=val_loader, num_epochs=EPOCHS, learning_rate=lr)


Epoch [1/50], Training Loss: 0.1430
Validation Loss: 0.1411
Epoch [2/50], Training Loss: 0.1430
Validation Loss: 0.1411
Epoch [3/50], Training Loss: 0.1429
Validation Loss: 0.1411
Epoch [4/50], Training Loss: 0.1429
Validation Loss: 0.1411
Epoch [5/50], Training Loss: 0.1429
Validation Loss: 0.1411
Epoch [6/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [7/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [8/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [9/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [10/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [11/50], Training Loss: 0.1429
Validation Loss: 0.1410
Epoch [12/50], Training Loss: 0.1428
Validation Loss: 0.1410
Epoch [13/50], Training Loss: 0.1428
Validation Loss: 0.1410
Epoch [14/50], Training Loss: 0.1428
Validation Loss: 0.1410
Epoch [15/50], Training Loss: 0.1428
Validation Loss: 0.1409
Epoch [16/50], Training Loss: 0.1428
Validation Loss: 0.1409
Epoch [17/50], Training Loss: 0.1

In [12]:
# Load the saved model for inference
def load_model(path: str):
    loaded_model = Autoencoder(input_dim=input_dim)
    loaded_model.load_state_dict(torch.load(path))
    loaded_model.eval()
    return loaded_model

In [13]:
# torch.save(model.state_dict(), 'AE_models/AE1.pth')

loaded_model = load_model('AE_models/AE1.pth')
ilicit = torch.tensor(df_merged.loc[df_merged['class'] == 0].drop(columns=['class', 'Time step', 'txId']).values, dtype=torch.float)
ilicit_data = CustomDataset(ilicit)
ilicit_loader = DataLoader(ilicit_data, shuffle=True)

reconstruction_loss = []
for inputs, targets in ilicit_loader:
    out = loaded_model(inputs)
    loss = criterion(out, targets)
    epoch_loss = loss.item() * inputs.size(0)
    reconstruction_loss.append(epoch_loss)
    
    


C:\Users\gsamp\AppData\Local\Temp\ipykernel_12932\3507107447.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float32), torch.tensor(sample, dtype=torch.float32)


OBTAIN THE LATENT REPRESENTATION OF THE AUTOENCODER

més fàcil que això simplement enxufar les daded a loaded_model.encoder()

In [14]:
#get only licit cases
x_norm = torch.tensor(df_merged.loc[df_merged['class'] == 0].drop(columns=["class", "txId", "Time step"]).values)
#scale
x_norm = preprocessing.MinMaxScaler().fit_transform(x_norm)
# Generate random indices
indices = torch.randperm(len(x_norm))[:4000]
# Sample elements using random indices
x_norm_tensor = x_norm[indices]

#get fraud cases
x_fraud = torch.tensor(df_merged.loc[df_merged['class'] == 1].drop(columns=["class", "txId", "Time step"]).values)
#scale
x_fraud_tensor = preprocessing.MinMaxScaler().fit_transform(x_fraud)

with torch.no_grad():
    norm_hid_rep = loaded_model.encoder(torch.tensor(x_norm_tensor[:4000]))
    fraud_hid_rep = loaded_model.encoder(x_fraud_tensor)

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
# Obtain latent space representation

#get only licit cases
x_norm = torch.tensor(df_merged.loc[df_merged['class'] == 0].drop(columns=["class", "txId", "Time step"]).values)
#scale
x_norm = preprocessing.MinMaxScaler().fit_transform(x_norm)
# Generate random indices
indices = torch.randperm(len(x_norm))[:4000]
# Sample elements using random indices
x_norm_tensor = x_norm[indices]

latent_data = CustomDataset(x_norm_tensor)
latent_loader = DataLoader(latent_data, shuffle=True)
licit_latent_representation  = []
fraud_latent_representation = []
with torch.no_grad():
    for licit_data, fraud_data in zip(latent_loader, ilicit_loader):
        #obtain the representation for the licit data
        inputs, _ = licit_data
        # inputs = inputs.view(inputs.size(0), -1)
        latent = loaded_model.encoder(inputs)
        licit_latent_representation.append(latent)
        
        #obtain the representation for the fraud data
        inputs, _ = fraud_data
        # inputs = inputs.view(inputs.size(0), -1)
        latent = loaded_model.encoder(inputs)
        fraud_latent_representation.append(latent)
        

norm_hid_rep= torch.cat(licit_latent_representation, dim=0)
print("Latent space representations shape:", norm_hid_rep.shape)


fraud_hid_rep = torch.cat(fraud_latent_representation, dim=0)
print("Latent space representations shape:", fraud_hid_rep.shape)

C:\Users\gsamp\AppData\Local\Temp\ipykernel_11488\3507107447.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sample, dtype=torch.float32), torch.tensor(sample, dtype=torch.float32)


Latent space representations shape: torch.Size([4000, 50])
Latent space representations shape: torch.Size([4000, 50])


In [ ]:
from sklearn.manifold import TSNE
import umap
def umap_plot(data, labels, save_path):
    umap_data = umap.UMAP(n_components=2, random_state=42).fit_transform(data)
    sns.scatterplot(x=umap_data[:, 0], y=umap_data[:, 1], hue=labels)
    plt.xlabel('UMAP component 1')
    plt.ylabel('UMAP component 2')
    plt.title('UMAP Plot')
    plt.savefig(save_path)
    plt.show()

ModuleNotFoundError: No module named 'umap'

In [ ]:
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis=0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Call umap_plot function
umap_plot(rep_x, rep_y, "latent_representation.png")

Exception in thread Thread-97 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 16: invalid start byte
c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of

KeyboardInterrupt: 